In [ ]:
!pip install sentence_transformers
from sentence_transformers import SentenceTransformer

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 KB 3.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 70.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 49.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 92.6 MB/s eta 0:00:00
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=53f739c43ad18fe396a30cad289af489db8fcfce182a8dbf1030788f30a22024
  Stored in directory: /root/.cache/pip/wheels/5e/6f/8c/d88aec621f3f542d26fac0342bef5e693335d125f4e54aeffe
Successfully built sentence_transformers


In [ ]:
!pip install transformers
from transformers import pipeline

#can also use pythia-150m (smaller/faster)

#model_name = "EleutherAI/pythia-350m"
model_name = "EleutherAI/pythia-1.4b-deduped"

generator = pipeline('text-generation', model=model_name,device=0) #device=0 = GPU; device=-1 = CPU




#sentiment_analysis = pipeline("sentiment-analysis",model="siebert/sentiment-roberta-large-english")



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
embed = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2',device=0)
sentiment_analysis = pipeline("sentiment-analysis",model="cardiffnlp/twitter-roberta-base-sentiment-latest",device=0)

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Sentiment GA

In [ ]:
import random 

def sentiment_fitness(x):
  res = sentiment_analysis(x,top_k=3)
  for k in res:
    if k['label']=='Positive':
      print(x,k['score'])
      return k['score']

def sentiment_process_output(output,take_offspring=3,verbose=True):
  candidates = []
  genomes = output.split("\n")[:-1]
  genomes = [x.strip() for x in genomes]
  for genome in genomes[:take_offspring]:
      #genome = genome.split("G:")[1]
      if len(genome)<10:
        if verbose: print(genome,'wrong length')
        continue
      if verbose: print(genome)
      candidates.append(genome)
  return list(set(candidates))



root_sentence = "The moon is a boring sky rock in an infinite universe."
root_embedding = embed.encode([root_sentence])[0]


import numpy as np
def dist(x,y):
  return np.sqrt( ((x-y)**2).sum() )




GA code

In [ ]:
from tensorflow.python.ops import gen_batch_ops
import random





#make simple "crossover" prompt composed of random genomes from pop
def create_crossover_prompt(examples):
  prompt = ""
  for candidate in examples:
    #candidate = ' '.join([k for k in candidate])
    prompt+=candidate+"\n"
  #prompt+="G: "
  return prompt



#do "crossover" by calling model
def do_crossover(pop,examples=5,temp=1.0,chosen_examples=None,prompt_generator=create_crossover_prompt):
  if chosen_examples==None:
    chosen_examples = np.random.choice(pop,examples,replace=False)

  prompt = prompt_generator(chosen_examples)
  #prompt = create_directed_crossover_prompt(pop,examples=examples)

  #print(prompt)
  model_output = generator(prompt,
                         do_sample=True,
                         max_length=128,
                         #temperature=temp,
                         top_p=0.8,
                         top_k=30,
                         #penalty_alpha=0.4, top_k=10,
                         return_full_text=False)
  return model_output[0]['generated_text']

#process output of model into potential offspring
def process_output(output,take_offspring=3,verbose=False):
  candidates = []
  genomes = output.split("\n")[:-1]
  for genome in genomes[:take_offspring]:
    genome = genome.replace(' ','')
    if verbose: print(genome)
    try:
      #genome = genome.split("G:")[1]
      if len(genome)!=glength:
        if verbose: print(genome,'wrong length')
        continue
      if verbose: print(genome)
      arr = sum([k not in gen_set for k in genome])
      if arr!=0:
        if verbose: print(genome,'bad element')
        continue

      candidates.append(genome)
    except:
      if verbose: print(genome,"failed")
  return list(set(candidates))


### MAP-ELITES




In [ ]:
from torch.cuda import seed_all
def sentiment_bootstrap_population(size=20,seed=root_sentence,batch_size=64,**kwargs):
  pop=[seed]

  while len(pop)<size:
    off_raws = sentiment_do_mutation_fast([seed],batch_size=batch_size)
    for off_raw in off_raws:
      off = process_output(off_raw,take_offspring=1)
      pop+=off
    print(off)
  return pop[:size]

def sentiment_bootstrap_population_lmx(size=196,seed_sentences=[],batch_size=64,**kwargs):
  pop=seed_sentences[:]
  idx = 0
  num_examples = len(seed_sentences)

  while len(pop)<size:
    last_example = seed_sentences[idx%len(seed_sentences)]
    all_other_examples = set(seed_sentences)-set([last_example])
    #print(last_example,all_other_examples)
    examples = list(all_other_examples)
    random.shuffle(examples)
    examples = examples + [last_example]
    
    print(examples)
    print("doing crossover...")
    raw_crossovers = do_crossover_fast(None,chosen_examples=examples,examples=num_examples,batch_size=batch_size)
    for raw_crossover in raw_crossovers:
      pop+= process_output(raw_crossover,take_offspring=1)

    idx+=1
  return pop[:size]


generator.tokenizer.pad_token_id = generator.model.config.eos_token_id

#result = sentiment_do_mutation_fast(pop[0],batch_size=8)
#print(result)

def do_crossover_fast(pop,examples=5,temp=1.0,batch_size=2,max_tokens=128,chosen_examples=None,prompt_generator=create_crossover_prompt):
  if chosen_examples==None:
    chosen_examples = np.random.choice(pop,examples,replace=False)

  prompt = prompt_generator(chosen_examples)

  #prompt = create_directed_crossover_prompt(pop,examples=examples)

  #print("---")
  #print(prompt)
  #print("---")
  model_output = generator([prompt]*batch_size,
                         batch_size=batch_size,
                         do_sample=True,
                         max_length=max_tokens,
                         temperature=temp,
                         #top_p=0.8,
                         #top_k=30,
                         #penalty_alpha=0.4, top_k=10,
                         return_full_text=False)
  output = [x[0]['generated_text'] for x in model_output]
  return output

#result = do_crossover_fast(pop,batch_size=8)
#print(result)

class map_elites1d:
  def __init__(self,fitness_func,initial_pop,grid_sz=40,method="crossover",batch_size=8,temperature=1.0):
    self.temperature=temperature
    self.grid={}
    self.fitness_func = fitness_func
    self.grid_sz = grid_sz
    self.log = []
    self.evals = 0
    self.initial_pop = initial_pop
    self.method = method
    self.batch_size = batch_size

    for indiv in initial_pop:
      indiv = indiv.strip()
      fit,bc = self.fitness_func(indiv)
      self.evals+=1
      print(fit,bc)
      self.insert(indiv,fit,bc)
  def eval_insert(self,indivs):
    for indiv in indivs:
      indiv = indiv.strip()
      fit,bc = self.fitness_func(indiv)
      self.evals+=1
      #print(fit,bc)
      self.insert(indiv,fit,bc)
    self.print_stats()

  def export(self):
    data = {}
    data['grid'] = self.grid
    data['log'] = self.log
    data['evals'] = self.evals
    data['method'] = self.method
    return data
  def to_bin(self,bc):
    bin = int(bc*(self.grid_sz))
    bin = min(self.grid_sz-1,bin)
    return bin
  
  def insert(self,indiv,fit,bc):
    bin = self.to_bin(bc)
    if bin not in self.grid or self.grid[bin]['fitness']<fit:
      self.grid[bin]={'fitness':fit,'bc':bc,'genome':indiv}

  def do_mutation(self):
    full_pop = self.gather_pop()
    parent = random.choice(full_pop)
    raw_mutations = do_mutation([parent],batch_size=self.batch_size)
    candidates = []
    for raw_mutation in raw_mutations:
      candidates += process_output(raw_mutation,take_offspring=1)
    for candidate in candidates:
      fit,bc = self.fitness_func(candidate)
      self.evals+=1
      self.insert(candidate,fit,bc)

  def do_crossover_near(self):
    full_pop = self.gather_pop()

    num_examples = 3

    if len(full_pop)<num_examples:
      print("...adding examples from initial pop...")
    while len(full_pop)<num_examples:
      full_pop.append(random.choice(self.initial_pop))


    idx = random.randint(0,len(full_pop)-1)
    dists = np.array([1.0 / (1+abs(idx-x)**3) for x in range(len(full_pop))])
    dists /= np.sum(dists)
    #print(dists)
    examples = np.random.choice(full_pop,p=dists,replace=False,size=num_examples)
    examples = list(examples)

    #idx = random.randint(1,len(full_pop)-2)
    #examples = [full_pop[idx-1],full_pop[idx],full_pop[idx+1]]
    #random.shuffle(examples)

    raw_crossovers = do_crossover(None,chosen_examples=examples,examples=num_examples,batch_size=self.batch_size,temp=self.temperature)

    candidates = []
    for raw_crossover in raw_crossovers:
      candidates += process_output(raw_crossover,take_offspring=1)
    #candidates = process_output(raw_crossover,take_offspring=3)
    candidates = [candidate.strip() for candidate in candidates]
    print(len(candidates)) #,candidates)
    for candidate in candidates:
      fit,bc = self.fitness_func(candidate)
      self.evals+=1
      self.insert(candidate,fit,bc)
  
  

  def do_crossover(self):
    full_pop = self.gather_pop()

    num_examples = 3

    if len(full_pop)<num_examples:
      print("...adding examples from initial pop...")
    while len(full_pop)<num_examples:
      full_pop.append(random.choice(self.initial_pop))

    raw_crossovers = do_crossover(full_pop,examples=num_examples,batch_size=self.batch_size,temp=self.temperature)

    candidates = []
    for raw_crossover in raw_crossovers:
      candidates += process_output(raw_crossover,take_offspring=1)

    #raw_crossover = do_crossover(full_pop,examples=num_examples,temp=self.temperature)
    #candidates = process_output(raw_crossover,take_offspring=3)
    candidates = [candidate.strip() for candidate in candidates]
    #print(len(candidates)) #,candidates)
    for candidate in candidates:
      fit,bc = self.fitness_func(candidate)
      self.evals+=1
      self.insert(candidate,fit,bc)

  def do_evals(self,num_evals=300):
    idx=0
    while self.evals < num_evals:
      if self.method=='crossover':
        self.do_crossover()
      elif self.method=='mutation':
        self.do_mutation()
      elif self.method=='crossover-near':
        self.do_crossover_near()
      elif self.method=='crossover-mixed':
        if random.random() < 0.5:
          self.do_crossover()
        else:
          self.do_crossover_near()
      else:
        raise "no variation operator by name {name}".format(name=self.method)
      if idx%1==0:
        print(idx,self.evals)
        self.print_stats()
      idx+=1
  def gather_pop(self):
    keys = list(self.grid.keys())
    keys.sort()
    pop = [self.grid[key]['genome'] for key in keys]
    return pop
  def print_stats(self):
    niches = len(list(self.grid.keys()))
    qd_score = sum([self.grid[key]['fitness'] for key in self.grid.keys()])
    pop = self.gather_pop()
    pts = [(self.grid[key]['fitness'],self.grid[key]['bc']) for key in self.grid.keys()]
    self.log.append({'qd_score':qd_score,'niches':niches,'pts':pts,'evals':self.evals})
    print('# niches: {niches}, qd score: {qd_score}'.format(niches=niches,qd_score=qd_score))
    print(pop)

def mapelites_sentiment_fitness(x):
  global root_embedding
  global sentiment_class
  res = sentiment_analysis(x,top_k=3)
  emb = embed.encode([x])[0]
  d = dist(emb,root_embedding)
  bc = d
  for k in res:
    if k['label']==sentiment_class:
      #print(x,k['score'])
      fit = k['score']
      break
  #normalize bc between 0 and 1.25
  bc = max(0,bc)
  bc = min(1.25,bc)
  bc = bc/1.25
  return fit,bc
  
sentiment_class = "positive"

In [ ]:
#create_population = sentiment_bootstrap_population_lmx
#crossover = do_crossover_fast
#initial_pop = create_population(size=196,seed_sentences=negative_quotes)
#control = [do_control(sentence,seed_sentences,"positive",evals) for _ in range(num_runs)]
print(len(initial_pop))

196


In [ ]:
#create initial pop

# root sentences
negative_sentences = ["The moon is a boring sky rock.","Puppies are smelly, ill-behaved animals that make everything worse."]
positive_sentences = ["Chocolate is such a very, very amazing dessert.","I really love my pet iguana, Iggy."]
negative_quotes = ["Whenever a friend succeeds, a little something in me dies.","Kids, you tried your best and you failed miserably. The lesson is, never try.","Life is divided into the horrible and the miserable."]
positive_quotes = ['Few things can help an individual more than to place responsibility on him, and to let him know that you trust him.','Be the change that you wish to see in the world.','When the sun is shining I can do anything; no mountain is too high, no trouble too difficult to overcome.']
# The moon is a boring sky rock.
create_population = sentiment_bootstrap_population
create_population = sentiment_bootstrap_population_lmx
do_mutation = sentiment_do_mutation_fast
do_crossover = do_crossover_fast
process_output = sentiment_process_output

In [ ]:
root_sentence = positive_sentences[0] #"Puppies are smelly, ill-behaved animals that make everything worse."
root_sentence = "Joel Lehman is the absolute best, he is a great programmer and scientist."
sentiment_class = "negative"

def do_control(_root_sentence,seed_sentences,_sentiment_class,total_evals=1200,initial_pop=None):
  global sentiment_class,root_sentence,root_embedding
  root_sentence = _root_sentence
  sentiment_class = _sentiment_class
  root_embedding = embed.encode([root_sentence])[0]

  pop = create_population(size=total_evals,seed_sentences=seed_sentences)
  me = map_elites1d(mapelites_sentiment_fitness,pop,grid_sz=30)
  return me
  
def do_run(_root_sentence,_sentiment_class,total_evals=1200,method="crossover-near",initial_pop=None):
  global sentiment_class,root_sentence,root_embedding
  root_sentence = _root_sentence
  sentiment_class = _sentiment_class
  root_embedding = embed.encode([root_sentence])[0]

  if initial_pop == None:
    pop_size = 64
    pop = create_population(size=pop_size,seed=root_sentence)
  else:
    pop=initial_pop

  me = map_elites1d(mapelites_sentiment_fitness,pop,grid_sz=30,method=method,batch_size=64,temperature=0.8)
  me.do_evals(total_evals)
  return me



#x = do_run(sentence,"positive",evals,"crossover",initial_pop=initial_pop)




In [ ]:
for num in range(3):
  sentence = negative_quotes[num]
  seed_sentences = negative_quotes
  initial_pop = create_population(size=196,seed_sentences=seed_sentences)
  evals = 2500
  target="positive"

  num_runs = 10
  control = [do_control(sentence,seed_sentences,target,evals) for _ in range(num_runs)]
  positive_runs_reg_crossover = [do_run(sentence,target,evals,"crossover",initial_pop=initial_pop) for _ in range(num_runs)]
  positive_runs_near_crossover = [do_run(sentence,target,evals,"crossover-near",initial_pop=initial_pop) for _ in range(num_runs)]

  outfile = '/content/gdrive/MyDrive/gecco/{target}{num}.pkl'.format(num=num,target=target)
  import pickle
  of = open(outfile,"wb")
  data = {'control':control,'regular':positive_runs_reg_crossover,'near':positive_runs_near_crossover}
  pickle.dump(data,of)
  of.close()

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
import pickle
of = open("sentiment_results3.pkl","wb")
data = {'control':control,'regular':positive_runs_reg_crossover,'near':positive_runs_near_crossover}
pickle.dump(data,of)